In [1]:
#import magic and libraries
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
#import data set and clean up
charity = pd.read_csv('Charity3.csv')
charity = charity.dropna(subset=['avg coll age'])
charity['avg coll age'] = charity['avg coll age'].convert_objects(convert_numeric=True)
charity['postcode'] = charity['postcode'].astype(str)
tax = pd.read_csv('2014tax2.csv')
tax['postcode'] = tax['postcode'].astype(str)
charity_new = pd.merge(charity, tax, on='postcode')
burs = pd.read_csv('abs2011combo.csv')
burs['postcode'] = burs['postcode'].astype(str)
charity_plus = pd.merge(charity_new, burs, on='postcode')
charity_plus.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


postcode  no of collectors  avg streets  new coll ratio  total collected  \
0      810                38         2.08            0.66          3782.75   
1      812                21         2.33            0.76          1759.10   
2      820                17         2.65            0.82          2360.65   
3      830                18         2.17            0.50          1464.35   
4      832                 9         1.89            0.56          1706.35   

   total donated  total received  avg collected  avg donated  avg received  \
0         349.25         4132.00          99.55         9.19        108.74   
1         200.00         1959.10          83.77         9.52         93.29   
2         100.00         2460.65         138.86         5.88        144.74   
3         400.00         1864.35          81.35        22.22        103.58   
4          57.00         1763.35         189.59         6.33        195.93   

     ...      ID_NS_OcMngr  ID_NS_OcProf  ID_NS_OcTechTrdW  ID_NS_OcComPerS  \
0    ...                 8            20                27               15   
1    ...                10             9                27               12   
2    ...                 6             4                13                6   
3    ...                11             4                28               17   
4    ...                 5             3                10                3   

   ID_NS_OcClericAdm  ID_NS_OcSalesWk  ID_NS_OcMacOp_Driv  ID_NS_OcLab  \
0                 15               10                  19           29   
1                 17               11                  12           18   
2                 19               10                   7           13   
3                 15                9                   4           11   
4                  8                5                   3            3   

   ID_NS_OcID_NS  ID_NS_Tot  
0             99        242  
1             69        185  
2             45        123  
3             55        154  
4             30         70  

[5 rows x 1209 columns]

In [3]:
#have a look at the data
charity_plus

postcode  no of collectors  avg streets  new coll ratio  total collected  \
0         810                38         2.08            0.66          3782.75   
1         812                21         2.33            0.76          1759.10   
2         820                17         2.65            0.82          2360.65   
3         830                18         2.17            0.50          1464.35   
4         832                 9         1.89            0.56          1706.35   
5         835                 3         2.33            1.00           189.75   
6         836                 4         2.25            1.00           467.30   
7         841                 1         2.00            1.00             0.00   
8         845                 2         2.00            1.00           211.05   
9         850                 2         2.00            1.00           305.65   
10        860                 1         3.00            0.00           203.30   
11        870                38         2.05            0.61          3711.00   
12        880                 3         2.00            1.00           125.00   
13       2000                 4         3.00            1.00          1216.50   
14       2007                 3         1.00            1.00           301.00   
15       2008                 3         3.67            0.67           194.00   
16       2010                11         1.73            0.82           605.25   
17       2011                 4         3.00            0.75           398.00   
18       2015                 6         1.00            0.67           804.35   
19       2016                 6         5.00            0.50           463.25   
20       2017                 6         6.50            0.67           627.10   
21       2018                12         2.33            0.67           657.00   
22       2019                 5         1.00            0.60           588.50   
23       2020                13         3.46            0.69           860.75   
24       2021                25         2.60            0.52          3187.30   
25       2022                14         2.71            0.79          1631.90   
26       2023                15         3.13            0.67          1726.00   
27       2024                13         5.38            0.92           906.00   
28       2025                13         2.85            0.54          1801.50   
29       2026                29         2.66            0.72          2455.70   
...       ...               ...          ...             ...              ...   
1698     7261                 2         4.00            0.50           140.00   
1699     7262                12         1.42            0.25           889.25   
1700     7263                 5         1.60            0.80           388.40   
1701     7265                 1         4.00            1.00           102.40   
1702     7267                 1         1.00            1.00            15.90   
1703     7268                 3         1.00            0.67           255.00   
1704     7270                15         1.20            0.80          1014.45   
1705     7275                 8         1.25            0.63           688.05   
1706     7277                28         1.25            0.68          2411.55   
1707     7290                13         1.08            0.54           669.80   
1708     7291                 2         1.00            0.50           160.00   
1709     7292                 3         1.00            0.00           473.90   
1710     7300                18         1.11            0.72          1063.40   
1711     7301                13         1.08            0.38           619.40   
1712     7302                 8         1.25            0.50           574.10   
1713     7303                 6         1.17            0.67           512.20   
1714     7304                15         1.20            0.67           989.25   
1715     7305                 3     

In [4]:
#preparing features for random forest
X = charity_plus.filter(regex="[^total received|^total collected|^total donated|^profitable|^avg received|^avg collected|^avg donated|^prof ratio|^no prof coll]")

In [5]:
#preparing target
y = np.asarray(charity_plus['profitable'], dtype="|S6")

In [36]:
#basic feature selection
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[  8.10000000e+02,   3.80000000e+01,   2.08000000e+00, ...,
          2.90000000e+01,   9.90000000e+01,   2.42000000e+02],
       [  8.12000000e+02,   2.10000000e+01,   2.33000000e+00, ...,
          1.80000000e+01,   6.90000000e+01,   1.85000000e+02],
       [  8.20000000e+02,   1.70000000e+01,   2.65000000e+00, ...,
          1.30000000e+01,   4.50000000e+01,   1.23000000e+02],
       ..., 
       [  7.33000000e+03,   1.60000000e+01,   1.31000000e+00, ...,
          1.10000000e+01,   2.10000000e+01,   4.90000000e+01],
       [  7.33100000e+03,   4.00000000e+00,   1.50000000e+00, ...,
          3.00000000e+00,   0.00000000e+00,   3.00000000e+00],
       [  7.46700000e+03,   5.00000000e+00,   1.40000000e+00, ...,
          0.00000000e+00,   6.00000000e+00,   1.00000000e+01]])

In [37]:
#create the model
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier(n_estimators=100, max_features=0.33, oob_score=True, random_state=1)
rfclf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.33, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [38]:
#check the oob score
rfclf.oob_score_

0.67534722222222221

In [8]:
#check the training accuracy
rfclf.score(X, y)

1.0

In [39]:
#have a look at which features were most important
importances = rfclf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rfclf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest - (too many to plot)
#plt.figure()
#plt.title("Feature importances")
#plt.bar(range(X.shape[1]), importances[indices],
#       color="r", yerr=std[indices], align="center")
#plt.xticks(range(X.shape[1]), indices)
#plt.xlim([-1, X.shape[1]])
#plt.show()

Feature ranking:
1. feature 0 (0.080881)
2. feature 7 (0.027171)
3. feature 15 (0.025801)
4. feature 8 (0.017694)
5. feature 6 (0.017131)
6. feature 12 (0.015466)
7. feature 9 (0.011512)
8. feature 4 (0.008920)
9. feature 128 (0.008833)
10. feature 3 (0.008338)
11. feature 125 (0.008021)
12. feature 2 (0.006524)
13. feature 999 (0.005562)
14. feature 817 (0.004836)
15. feature 1008 (0.004625)
16. feature 11 (0.004347)
17. feature 1145 (0.003348)
18. feature 130 (0.003202)
19. feature 1026 (0.003164)
20. feature 815 (0.003145)
21. feature 67 (0.003116)
22. feature 126 (0.003064)
23. feature 816 (0.002990)
24. feature 1006 (0.002948)
25. feature 62 (0.002884)
26. feature 60 (0.002644)
27. feature 814 (0.002610)
28. feature 63 (0.002536)
29. feature 829 (0.002447)
30. feature 845 (0.002427)
31. feature 987 (0.002422)
32. feature 1018 (0.002417)
33. feature 178 (0.002397)
34. feature 1001 (0.002368)
35. feature 983 (0.002328)
36. feature 127 (0.002324)
37. feature 947 (0.002254)
38. featur

In [10]:
#remember which features correspond to which 
features = X.columns.tolist()
features[951]

'F_85ov_Volunteer'

In [11]:
#1. is avg tax income, 2. avg income, 3. is gender, 4. is whether new or acquired collector, 5. is postcode
#this makes a lot of sens[]

col_list = X[indices[0:9]]


In [12]:
col_list

avg tax income    avg income  avg gross tax paid    avg salary  \
0        72332.05379   75201.57679        16109.736830   70174.69852   
1        70571.93514   73118.01099        15274.254690   69415.30501   
2        84160.95487   87489.84599        20760.580330   77015.86339   
3        71291.49362   74124.89540        15479.880400   71260.41744   
4        75671.92639   78607.22006        17044.260960   75603.31895   
5        80743.81995   83631.26382        19160.969060   76117.25214   
6        76981.05545   79865.39459        17616.236540   76208.36337   
7        71721.16064   74351.24498        15710.461850   72563.32189   
8        63249.40000   66315.58491        12701.354720   63829.14575   
9        64606.54167   66888.91701        13168.598260   62357.34613   
10       66437.70143   70849.32143        13763.762860   67010.99704   
11       66100.15483   68752.56318        13837.191420   64164.21201   
12       85724.25600   88342.39800        21175.454910   82868.71339   
13       86644.86732   90029.27358        24659.512170   68324.53158   
14       66089.59117   68641.78439        14423.002560   63406.80715   
15       69572.54394   72288.16173        15560.216410   65432.75240   
16       93319.31069   97212.71674        24859.085310   83607.30431   
17      106234.29580  111004.61120        30327.997180   87041.84259   
18       81095.37875   84749.62658        19394.310990   78642.97207   
19       79465.95438   82890.14904        19138.206230   73037.29836   
20       73606.78650   77156.78509        16627.235640   72972.70994   
21       64750.03044   67812.56919        13555.187290   64027.76782   
22       70819.76172   74286.23297        15641.175370   69395.84363   
23       65553.16964   68660.91619        13677.082730   65175.90190   
24      133380.30480  139050.28390        41884.570790  104154.45700   
25       97381.28031  101789.96330        27034.068760   85788.51544   
26      188492.00280  198061.62910        66074.159080  114546.96710   
27      135745.01100  141632.49750        43102.363410  103617.10620   
28      161282.76240  169595.79560        53976.449490  110457.31180   
29      104357.25730  109373.65190        29755.710090   86948.96874   
...              ...           ...                 ...           ...   
1698     54796.54688   58384.01563        10233.906250   44453.42574   
1699     56467.80398   58924.62175        10788.908120   49494.25865   
1700     54751.84574   57288.96809        10190.547870   47784.22143   
1701     50914.62791   53500.58140         8650.895349   48025.25397   
1702     55179.99388   57244.26300        10045.207950   52567.64138   
1703     59444.85060   61761.44096        11564.853010   55973.18384   
1704     58644.77147   60720.37193        11576.404140   53252.49914   
1705     59479.29339   61932.66281        11769.474380   53592.79063   
1706     61123.47342   63612.02841        12235.194780   58171.32461   
1707     53591.45316   55646.84532         9515.593682   51897.25116   
1708     55907.22727   58207.14463        10254.801650   53522.87500   
1709     55051.21311   57766.55328        10288.975410   54460.27273   
1710     56267.70014   58532.05667        10438.562410   52891.44340   
1711     57430.27173   59891.84377        11061.383590   50575.13525   
1712     57094.44553   59501.19837        11013.507320   49862.18750   
1713     60610.32558   63078.63531        12365.912260   53044.66626   
1714     54162.03570   56894.54248         9879.322938   50379.99457   
1715     52440.06301   54627.62740         9135.331507   50911.70245   
1716     55261.96536   57840.64307        10182.286900   53541.60209   
1717     59496.33702   62098.23456        11663.791050   56726.03775   
1718     56956.10710   59463.34934        10835.177740   53333.66644   
1719     56475.24105   58838.55127        10563.192540   54159.41194   
1720     61394.59948   63959.06704        12399.620370   57782.47276   
1721     59571.21863   61927.51965        1171

In [16]:
#map my predictions back to the df as a column
col_list['prediction'] = rfclf.predict(X)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [23]:
col_list['profitable'] = charity_plus['profitable']

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
#create smaller df for export for visualisation
final_cols = col_list
#final_cols = charity_plus['prediction', col_list]
#col_l2 = X[250,938,937,37,40]
final_cols

avg tax income    avg income  avg gross tax paid    avg salary  \
0        72332.05379   75201.57679        16109.736830   70174.69852   
1        70571.93514   73118.01099        15274.254690   69415.30501   
2        84160.95487   87489.84599        20760.580330   77015.86339   
3        71291.49362   74124.89540        15479.880400   71260.41744   
4        75671.92639   78607.22006        17044.260960   75603.31895   
5        80743.81995   83631.26382        19160.969060   76117.25214   
6        76981.05545   79865.39459        17616.236540   76208.36337   
7        71721.16064   74351.24498        15710.461850   72563.32189   
8        63249.40000   66315.58491        12701.354720   63829.14575   
9        64606.54167   66888.91701        13168.598260   62357.34613   
10       66437.70143   70849.32143        13763.762860   67010.99704   
11       66100.15483   68752.56318        13837.191420   64164.21201   
12       85724.25600   88342.39800        21175.454910   82868.71339   
13       86644.86732   90029.27358        24659.512170   68324.53158   
14       66089.59117   68641.78439        14423.002560   63406.80715   
15       69572.54394   72288.16173        15560.216410   65432.75240   
16       93319.31069   97212.71674        24859.085310   83607.30431   
17      106234.29580  111004.61120        30327.997180   87041.84259   
18       81095.37875   84749.62658        19394.310990   78642.97207   
19       79465.95438   82890.14904        19138.206230   73037.29836   
20       73606.78650   77156.78509        16627.235640   72972.70994   
21       64750.03044   67812.56919        13555.187290   64027.76782   
22       70819.76172   74286.23297        15641.175370   69395.84363   
23       65553.16964   68660.91619        13677.082730   65175.90190   
24      133380.30480  139050.28390        41884.570790  104154.45700   
25       97381.28031  101789.96330        27034.068760   85788.51544   
26      188492.00280  198061.62910        66074.159080  114546.96710   
27      135745.01100  141632.49750        43102.363410  103617.10620   
28      161282.76240  169595.79560        53976.449490  110457.31180   
29      104357.25730  109373.65190        29755.710090   86948.96874   
...              ...           ...                 ...           ...   
1698     54796.54688   58384.01563        10233.906250   44453.42574   
1699     56467.80398   58924.62175        10788.908120   49494.25865   
1700     54751.84574   57288.96809        10190.547870   47784.22143   
1701     50914.62791   53500.58140         8650.895349   48025.25397   
1702     55179.99388   57244.26300        10045.207950   52567.64138   
1703     59444.85060   61761.44096        11564.853010   55973.18384   
1704     58644.77147   60720.37193        11576.404140   53252.49914   
1705     59479.29339   61932.66281        11769.474380   53592.79063   
1706     61123.47342   63612.02841        12235.194780   58171.32461   
1707     53591.45316   55646.84532         9515.593682   51897.25116   
1708     55907.22727   58207.14463        10254.801650   53522.87500   
1709     55051.21311   57766.55328        10288.975410   54460.27273   
1710     56267.70014   58532.05667        10438.562410   52891.44340   
1711     57430.27173   59891.84377        11061.383590   50575.13525   
1712     57094.44553   59501.19837        11013.507320   49862.18750   
1713     60610.32558   63078.63531        12365.912260   53044.66626   
1714     54162.03570   56894.54248         9879.322938   50379.99457   
1715     52440.06301   54627.62740         9135.331507   50911.70245   
1716     55261.96536   57840.64307        10182.286900   53541.60209   
1717     59496.33702   62098.23456        11663.791050   56726.03775   
1718     56956.10710   59463.34934        10835.177740   53333.66644   
1719     56475.24105   58838.55127        10563.192540   54159.41194   
1720     61394.59948   63959.06704        12399.620370   57782.47276   
1721     59571.21863   61927.51965        1171

In [18]:
#exportpostcodedf = charity_plus[col_list]
#exportpostcodedf

In [25]:
col_list.to_csv('exportpostcodelevel2805.csv')

In [31]:
col_list.profitable.dtype

dtype('int64')

In [33]:
from sklearn.metrics import confusion_matrix
col_list['profitable'] = col_list['profitable'].astype(str)
confusion_matrix(col_list['profitable'], col_list['prediction'])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


array([[805,   0],
       [  0, 923]])